### Import modules

In [ ]:
#-----------------
# Standard imports
#-----------------
import numpy as np # for arrays
import matplotlib.pyplot as plt # for plotting
from sklearn.decomposition import PCA # for PCA for normalization
from scipy.spatial import distance_matrix
import os # for dealing with directories
from os import listdir # for retrieving files from directory
from os.path import isfile, join # for retrieving files from directory
from sklearn.manifold import MDS # for MDS
import pandas as pd # for loading in colors csv
from scipy.spatial import ConvexHull # for convex hull
import seaborn as sns # for plotting
import math # for isnan

#---------------------------
# The ECT packages we'll use
#---------------------------
from ect import ECT, EmbeddedGraph # for calculating ECTs # pip install ect

#---------------------------
# The PHATE packages we'll use
#---------------------------
import phate # for using PHATE # pip install --user phate
import scprep # for using PHATE

### Define functions

In [ ]:
def normalize(shape):
    """
    input: ordered coordinates of a 2D closed contour
    output: 2D array, coordinates origin centered on centroid and longest radius = 1
    """
    pca = PCA(n_components=2) # initiate PCA
    pca.fit_transform(shape) # fit PCA to leaf data to find longest axis
    pca_scores = pca.transform(shape) # retrieve PCA scores of leaf

    # return leaf normalized by longest axis to 1, zero centered
    return pca_scores/(np.max(pca_scores[:,0])-np.min(pca_scores[:,0]))

In [ ]:
def get_ect(normal_sh, dir_num, thresh_num, global_bound_radius):
    """
    inputs: ordered coordinates of a normalized 2D closed contour, longest diameter = 1
    dir_num = number of directional axes
    thresh_num = number of thresholds
    global_bound_radius = half of the longest normalized diameter
    output: ECT as 2D array
    """
    G = EmbeddedGraph() # initiate an embedded graph

    valuesX = normal_sh[:,0] # isolate x vals
    valuesY = normal_sh[:,1] # isolate y vals
    for i in range(np.shape(normal_sh)[0]): # create nodes
        G.add_node(i,valuesX[i],valuesY[i])
    for i in range(np.shape(normal_sh)[0]-1): # create edges
        G.add_edge(i, i+1)
    G.add_edge(0,np.shape(normal_sh)[0]-1) # add final edge between start and end

    myect = ECT(num_dirs = dir_num, num_thresh=thresh_num) # intiate ECT
    myect.set_bounding_radius(global_bound_radius) # set bounding radius
    myect.calculateECT(G) # calculate ECT on embedded graph

    return myect.get_ECT() # retrieve ECT, 2D array, [axes, thresh]

In [ ]:
def ect_diff(ect1, ect2,dir_num):
    """
    inputs: two, 2D arrays of ECTs and number of directional axes
    output: the minimum sum diff between the two ECT arrays across different orientations
    """
    diffs = [] # store differences 
    for i in range(dir_num): # for the number of directional axes
        roll_ect = np.roll(ect1,i,axis=0) # roll the array by 1 along directional axes
        diff_ect = np.abs(ect2 - roll_ect) # difference between the two arrays
        diffs.append(np.sum(diff_ect)) # store the sum of the differences
    return np.min(diffs) # return minimum difference

In [ ]:
def poly_area(x,y):
    """
    define a function to calculate the area of a polygon using the shoelace algorithm
    inputs: separate numpy arrays of x and y coordinate values
    outputs: the area of the polygon
    """
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

_______

# Calculate the Euler Characteristic Transform (ECT) for leaf shapes

_______

***The method:***

Learn more about the Euler Charactertistic Transform from the manuscript below:

Elizabeth Munch (2023) **[An Invitation to the Euler Characteristic Transform](https://arxiv.org/abs/2310.10395)**, *arXiv*

In this Jupyter notebook tutorial, we use the [`ect` module](https://munchlab.github.io/ect/) in python developed by Elizabeth Munch and her lab.

***The data:***

The following data was stored in a folder called `2017_leaves` in the working directory. The data can be downloaded from:

Wang, Elena; Willey, Nathan; Chitwood, Dan (2024). [Modified leaf shape contour data](https://doi.org/10.6084/m9.figshare.25435936.v1). figshare. Dataset. https://doi.org/10.6084/m9.figshare.25435936.v1

### Define datasets and groups

In [ ]:
# a list of folder directories of datasets to analyze
datasets = ["Alstroemeria",
            "Apple",
            "Arabidopsis",
            "Arabidopsis_asymmetry",
            "Brassica/napus",
            "Brassica/rapa",
            "Coleus/Co1",
            "Coleus/Co2",
            "Coleus/Co3",
            "Coleus/Co4",
            "Coleus/Co5",
            "Coleus/Co6",
            "Cotton",
            "Grape/Domesticated_grape/GrapeJune1.2011",
            "Grape/Domesticated_grape/GrapeMay28.2011",
            "Grape/Domesticated_grape/GrapeMay29.2011",
            "Grape/Domesticated_grape/GrapeMay30.2011",
            "Grape/Domesticated_grape/GrapeMay31.2011",
            "Grape/Geneva_2013/Folder1_Jose",
            "Grape/Geneva_2013/Folder2_Tommy",
            "Grape/Geneva_2013/Folder3_Danny",
            "Grape/Geneva_2013/Folder4_Quaneisha",
            "Grape/Geneva_2013/Folder5_Susan",
            "Grape/Geneva_2013/Folder6_Darren",
            "Grape/Geneva_2013/Folder7_Jose2",
            "Grape/Geneva_2013/Folder8_Susan2",
            "Grape/Geneva_2013/Folder9_Tommy2",
            "Grape/Geneva_2013/Folder10_Quaneisha2",
            "Grape/Geneva_2013/Folder11_Darren2",
            "Grape/Geneva_2013/Folder12_Danny",
            "Grape/Geneva_2015/060815",
            "Grape/Geneva_2015/060915",
            "Grape/Geneva_2015/061015",
            "Grape/Geneva_2016/Dan_Geneva_2016",
            "Grape/Geneva_2016/MF_Geneva_2016",
            "Grape/MtVernon_scans/2014_MtVernon_Scans",
            "Grape/MtVernon_scans/2016_MtVernon_Scans",
            "Grass",
            "Ivy",
            "Leafsnap",
            "Passiflora",
            "Pepper",
            "Potato",
            "Tomato_asymmetry",
            "Tomato_BILs/B1",
            "Tomato_BILs/B2",
            "Tomato_BILs/B3",
            "Tomato_chamber/C1",
            "Tomato_chamber/C2",
            "Tomato_chamber/C3",
            "Tomato_chamber/C4",
            "Tomato_chamber/C5",
            "Tomato_entire",
            "Tomato_field/F1",
            "Tomato_field/F2",
            "Tomato_wild/WA",
            "Tomato_wild/WB",
            "Transect/Batch_1_W",
            "Transect/Batch_2_B",
            "Viburnum/V1",
            "Viburnum/V2",
           ]

# make a list of overall groups
groups = [] # store group names

for i in datasets: # for each dataset

    if ("Tomato" in i) & ("/" in i):
        groups.append(i[:i.index('_')]) # then only take up to the underscore
    
    elif "/" in i: # if there are subfolders
        groups.append(i[:i.index('/')]) # then only take the name of the first folder
    elif "_" in i: # if there is a redundant folder with "_" like Arabidopsis
        groups.append(i[:i.index('_')]) # then only take up to the underscore
    else:
        groups.append(i) # otherwise just take the whole name

### Create a dataframe of all files

Create a pandas dataframe of file names, groups, and datasets

In [ ]:
file_names = [] # store file names
dataset = [] # store dataset names
group = [] # store group names

for i in range(len(datasets)): # for each dataset
    
    curr_dataset = datasets[i] # get current dataset
    curr_group = groups[i] # get current group
    data_dir = "./2017_leaves/"+curr_dataset+"/" # set data directory
    files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names
    #file_names.remove('.DS_Store') # remove .DS_Store file
    files.sort() # sort the list of file names

    for j in range(len(files)): # for each file
        
        file_names.append("./2017_leaves/"+curr_dataset+"/"+files[j]) # store file names
        dataset.append(curr_dataset) # store datasets
        group.append(curr_group) # store group
        
# create a pandas dataframe of all file info
lf_df = pd.DataFrame(
    {"group":group,
     "dataset":dataset,
     "file":file_names,
    })

# print out number of each dataset
lf_df["group"].value_counts()

### Create a downsampled dataset (or not)

The code below will either downsample or not. If you don't want to downsample, select downsample as `False`. If you do want to downsample, select `True` and select the random number you want to select from each clade.

In [ ]:
downsample = True # do you want to downsample, True or False
rand_num = 800 # select number to randomly sample
df_list = [] # a list to store pandas dataframes
rand_state = 42 # set the random state

if downsample==False:
    rand_df = lf_df

else:
    for i in lf_df["group"].unique(): # for each group
        curr_group = lf_df[lf_df["group"]==i] # select all samples of current group
        df_list.append(curr_group.sample(n=rand_num, random_state=rand_state)) # select random rows for the current group
        
    rand_df = pd.concat(df_list).reset_index() # concatenate list of dataframes together

print(len(rand_df)) # print the overall length of the selected cells

### Analyze solidity and aspect ratio

Calculate solidity and aspect ratio

In [ ]:
solidity = [] # store solidity values
wl_ratio = [] # store width-to-length ratio values

for i in range(len(rand_df)): # for each of the randomly selected leaves
    
    curr_file = rand_df["file"][i] # get current file
     
    # calculate solidity
    curr_cell = np.load(curr_file) # load current leaf
    hull = ConvexHull(curr_cell) # calculate convex hull of current leaf
    vertices = hull.vertices # isolate vertex indices of convex hull
    area = poly_area(curr_cell[:,0], curr_cell[:,1]) # calculate area
    convex_area = poly_area(curr_cell[vertices,0], curr_cell[vertices,1]) # calculate convex area
    solidity.append( area / convex_area ) # calculate solidity and store

    # calculate length-to-width ratio
    pca = PCA(n_components=2) # initiate PCA
    pca.fit_transform(curr_cell) # fit PCA to cell data to find longest axis
    pca_scores = pca.transform(curr_cell) # retrieve PCA scores of cell
    length = np.max(pca_scores[:,0])-np.min(pca_scores[:,0]) # get length
    width = np.max(pca_scores[:,1])-np.min(pca_scores[:,1]) # get width
    wl_ratio.append(width/length) # store length-to-width ratio
    
rand_df["solidity8"] = np.array(solidity)**8 # add solidity to random leaves
rand_df["wl_ratio"] = wl_ratio # add width-to-length ratio to random leaves

Plot solidity and aspect ratio

In [ ]:
ax = sns.scatterplot(data=rand_df,
                x="wl_ratio",
                y="solidity8",
                hue="group",
                s=5,
                linewidth=0,
                alpha=1
               )
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.title("Solidity^8 vs width-to-length ratio\nby group")

Plot solidity and aspect ratio with leaf outlines

In [ ]:
rand_num = 200 # set number of random cells to visualize
scale = 0.02 # set scale of leaves
rand_state = 42 # set the random state

rand_plot_df = rand_df.sample(n=rand_num, random_state=rand_state, ignore_index=True) # select random leaves

for i in range(len(rand_plot_df)): # for each of the randomly selected leaves

    curr_lf = np.load(rand_plot_df["file"][i]) # get current file
    pca = PCA(n_components=2) # initiate PCA
    pca.fit_transform(curr_lf) # fit PCA to cell data to find longest axis
    pca_scores = pca.transform(curr_lf) # retrieve PCA scores of leaf
    
    curr_lfx = pca_scores[:,0]*scale+rand_plot_df["wl_ratio"].iloc[i]
    curr_lfy = pca_scores[:,1]*scale+rand_plot_df["solidity8"].iloc[i]
    
    plt.fill(curr_lfx, curr_lfy, lw=0, c="k", alpha=0.3)
    plt.xlabel("wl_ratio")
    plt.ylabel("solidity8")
    plt.title("Solidity^8 vs width-to-length ratio\nshowing leaf shape")

### Calculate ECTs for each leaf

We need a folder to store the `ect` files. The code below will create the `ect_files` folder if it does not already exist.

If you already created the `ect` files and have the `ect_files` folder, then you don't need to run the following code.


In [ ]:
path = "ect_files"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

In [ ]:
rand_df

In [ ]:
num_dir=32 # set number of directional axes
num_thresh=48 # set number of thresholds each axis
rad = 0.5 # set the radius

# create array to store ECT outputs
# number of ECTs x num directional axes x number of thresholds
ect_arr = np.zeros((len(rand_df),num_dir,num_thresh))

for i in range(len(rand_df)): # for each cell
    if i%100==0: # print if divisible by 100
        print(i)
    shape = np.load(rand_df["file"].iloc[i]) # get the current shape
    ect_arr[i,:,:] = get_ect(normal_sh=normalize(shape), # normalize the shape using PCA, save to ect_arr
           dir_num=num_dir, # calculate the ECT
           thresh_num=num_thresh,
           global_bound_radius=rad)
    np.save("./ect_files/"+rand_df["group"].iloc[i]+"_"+str(i)+".npy",ect_arr[i,:,:]) # save npy file

### Plot leaves next to ECTs

In [ ]:
rand_num = 40 # set number of random leaves to visualize
rand_state = 42 # set the random state

rand_ect_df = rand_df.sample(n=rand_num, random_state=rand_state) # select random leaves

counter = 1 # counter for plt.subplot
plt.figure(figsize=(10,7))

for i in rand_ect_df.index: # for each of the randomly chosen indices
    
    plt.subplot(8,10,counter)
    curr_file = rand_ect_df["file"][i] # get current file
    curr_leaf = np.load(curr_file) # load current leaf
    pca = PCA(n_components=2) # initiate PCA
    pca.fit_transform(curr_leaf) # fit PCA to leaf data to find longest axis
    pca_scores = pca.transform(curr_leaf) # retrieve PCA scores of leaf
    plt.fill(pca_scores[:,0], pca_scores[:,1], lw=0, c="k", alpha=0.5)
    plt.axis("off")
    plt.gca().set_aspect("equal")
    
    counter+=1
    
    plt.subplot(8,10,counter)
    plt.imshow(ect_arr[i])
    plt.axis("off")
    plt.gca().set_aspect("equal")
    
    counter+=1
    
plt.tight_layout()


After saving `ect` representations of each leaf above to the folder `ect_files`, you will be able to skip the above section. But it is important we save the associated dataframe `rand_df` as well. We will save this file as `leaf_df.csv`.

In [ ]:
rand_df.to_csv("leaf_df.csv", index=False)

______

# PHATE (Potential of Heat-diffusion for Affinity-based Trajectory Embedding)

We will be using [PHATE](https://github.com/KrishnaswamyLab/PHATE) to visualize the manifold underlying pavement cell shapes. You can learn more about PHATE from the following reference:

Moon, van Dijk, Wang, Gigante et al. **[Visualizing Transitions and Structure for Biological Data Exploration](https://www.nature.com/articles/s41587-019-0336-3)**. 2019. *Nature Biotechnology*.

_____

You can skip to this section if you have already saved `ect` representations of the leaf shapes as `.npy` files in the folder `ect_files`, as described above. 

If you are skipping to this section, it is a good idea to read in the associated datafile `leaf_df.csv` again (which was saved above).

In [ ]:
leaf_df = pd.read_csv("leaf_df.csv")

Before using PHATE, we will need to flatten the ect array

In [ ]:
# flatten the cell ECT data to a 2D array by sample
flat_leaf_ect = np.reshape(ect_arr,
                             (np.shape(ect_arr)[0],
                              np.shape(ect_arr)[1]*np.shape(ect_arr)[2])
                            )

The following is from ***[read the docs](https://phate.readthedocs.io/en/stable/)*** for PHATE:

- n_components (int, optional, default: 2) – number of dimensions in which the data will be embedded
- knn (int, optional, default: 5) – number of nearest neighbors on which to build kernel
- decay (int, optional, default: 40) – sets decay rate of kernel tails. If None, alpha decaying kernel is not used
- n_landmark (int, optional, default: 2000) – number of landmarks to use in fast PHATE
- t (int, optional, default: 'auto') – power to which the diffusion operator is powered. This sets the level of diffusion. If ‘auto’, t is selected according to the knee point in the Von Neumann Entropy of the diffusion operator
- gamma (float, optional, default: 1) – Informational distance constant between -1 and 1. gamma=1 gives the PHATE log potential, gamma=0 gives a square root potential.
- n_pca (int, optional, default: 100) – Number of principal components to use for calculating neighborhoods. For extremely large datasets, using n_pca < 20 allows neighborhoods to be calculated in roughly log(n_samples) time.
- mds_solver ({'sgd', 'smacof'}, optional (default: 'sgd')) – which solver to use for metric MDS. SGD is substantially faster, but produces slightly less optimal results. Note that SMACOF was used for all figures in the PHATE paper.
- knn_dist (string, optional, default: 'euclidean') – recommended values: ‘euclidean’, ‘cosine’, ‘precomputed’ Any metric from scipy.spatial.distance can be used distance metric for building kNN graph. Custom distance functions of form f(x, y) = d are also accepted. If ‘precomputed’, data should be an n_samples x n_samples distance or affinity matrix. Distance matrices are assumed to have zeros down the diagonal, while affinity matrices are assumed to have non-zero values down the diagonal. This is detected automatically using data[0,0]. You can override this detection with knn_dist=’precomputed_distance’ or knn_dist=’precomputed_affinity’.
- knn_max (int, optional, default: None) – Maximum number of neighbors for which alpha decaying kernel is computed for each point. For very large datasets, setting knn_max to a small multiple of knn can speed up computation significantly.
- mds_dist (string, optional, default: 'euclidean') – Distance metric for MDS. Recommended values: ‘euclidean’ and ‘cosine’ Any metric from scipy.spatial.distance can be used. Custom distance functions of form f(x, y) = d are also accepted
- mds (string, optional, default: 'metric') – choose from [‘classic’, ‘metric’, ‘nonmetric’]. Selects which MDS algorithm is used for dimensionality reduction
- n_jobs (integer, optional, default: 1) – The number of jobs to use for the computation. If -1 all CPUs are used. If 1 is given, no parallel computing code is used at all, which is useful for debugging. For n_jobs below -1, (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one are used
- random_state (integer or numpy.RandomState, optional, default: None) – The generator used to initialize SMACOF (metric, nonmetric) MDS If an integer is given, it fixes the seed Defaults to the global numpy random number generator
- verbose (int or boolean, optional (default: 1)) – If True or > 0, print status messages
- potential_method (deprecated.) – Use gamma=1 for log transformation and gamma=0 for square root transformation.
- kwargs (additional arguments for graphtools.Graph) –

### PHATE in 2 dimensions

In [ ]:
# create the PHATE embedding and transform
phate_operator = phate.PHATE(n_components=2)
phate_leaf = phate_operator.fit_transform(flat_leaf_ect)

Add the PHATE axes to the dataframe for plotting

In [ ]:
leaf_df["2Dphate1"] = phate_leaf[:,0]
leaf_df["2Dphate2"] = phate_leaf[:,1]

Head the dataframe so we know what we are working with

In [ ]:
leaf_df.head()

Create a PHATE plot in 2D

In [ ]:
plt.figure(figsize=(10,4)) # set figure size
pt_size = 1 # set point size
pal = "inferno" # set palette

plt.subplot(1,3,1)
sns.scatterplot(data=leaf_df, x="2Dphate1", y="2Dphate2", hue="group", legend=False, s=pt_size)
plt.title("PHATE1 and PHATE2\nby group")

plt.subplot(1,3,2)
sns.scatterplot(data=leaf_df, x="2Dphate1", y="2Dphate2", hue="wl_ratio", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE2\nby width-to-length ratio")

plt.subplot(1,3,3)
sns.scatterplot(data=leaf_df, x="2Dphate1", y="2Dphate2", hue="solidity8", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE2\nby solidity")

plt.tight_layout()

### PHATE in 3 dimensions

In [ ]:
# create the PHATE embedding and transform
phate_operator = phate.PHATE(n_components=3)
phate_leaf = phate_operator.fit_transform(flat_leaf_ect)

Add the PHATE axes to the dataframe for plotting

In [ ]:
leaf_df["3Dphate1"] = phate_leaf[:,0]
leaf_df["3Dphate2"] = phate_leaf[:,1]
leaf_df["3Dphate3"] = phate_leaf[:,2]

Head the dataframe so we know what we are working with

In [ ]:
leaf_df.head()

Create a 3D animation by major clade

In [ ]:
sample_labels = leaf_df["group"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_leaf, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", filename="leaves_group.gif")


Create a 3D animation by width-to-length ratio

In [ ]:
sample_labels = leaf_df["wl_ratio"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_leaf, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", 
                             filename="leaves_aspect_ratio.gif", cmap="inferno")

Create a 3D animation by solidity

In [ ]:
sample_labels = leaf_df["solidity8"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_leaf, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", 
                             filename="leaves_solidity.gif", cmap="inferno")

Create a static subplot of 3D PHATE results

In [ ]:
plt.figure(figsize=(20,20)) # set figure size
pt_size = 10 # set point size
pal = "inferno" # set palette

#####################
# plot by major clade
#####################

plt.subplot(3,3,1)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate2", hue="group", legend=False, s=pt_size)
plt.title("PHATE1 and PHATE2\nby group")

plt.subplot(3,3,2)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate3", hue="group", legend=False, s=pt_size)
plt.title("PHATE1 and PHATE3\nby group")

plt.subplot(3,3,3)
sns.scatterplot(data=leaf_df, x="3Dphate2", y="3Dphate3", hue="group", legend=False, s=pt_size)
plt.title("PHATE2 and PHATE3\nby group")

###############################
# plot by width-to-length ratio
###############################

plt.subplot(3,3,4)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate2", hue="wl_ratio", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE2\nby width-to-length ratio")

plt.subplot(3,3,5)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate3", hue="wl_ratio", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE3\nby width-to-length ratio")

plt.subplot(3,3,6)
sns.scatterplot(data=leaf_df, x="3Dphate2", y="3Dphate3", hue="wl_ratio", legend=False, s=pt_size, palette=pal)
plt.title("PHATE2 and PHATE3\nby width-to-length ratio")

##################
# plot by solidity
##################

plt.subplot(3,3,7)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate2", hue="solidity8", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE2\nby solidity")

plt.subplot(3,3,8)
sns.scatterplot(data=leaf_df, x="3Dphate1", y="3Dphate3", hue="solidity8", legend=False, s=pt_size, palette=pal)
plt.title("PHATE1 and PHATE3\nby solidity")

plt.subplot(3,3,9)
sns.scatterplot(data=leaf_df, x="3Dphate2", y="3Dphate3", hue="solidity8", legend=False, s=pt_size, palette=pal)
plt.title("PHATE2 and PHATE3\nby solidity")

In order to test the effects of rotation of the contour, we will translate each ECT along its direction axis (which is axis=0). Because we had 32 directions, we will use increments of 8 and the `np.roll` function 3 times (th fourth translation will be the original ECT image). We will save the different rotated/translated ECT images to arrays and compare their placement in PHATE.

In [ ]:
ect_rot1 = np.zeros(np.shape(ect_arr)) # create array to store first rotation
ect_rot2 = np.zeros(np.shape(ect_arr)) # create array to store second rotation
ect_rot3 = np.zeros(np.shape(ect_arr)) # create array to store third rotation

for i in range(np.shape(ect_arr)[0]): # for each cell ECT

    if i%1000==0: # print if i divisible by 1000
        print(i)

    curr_cell = ect_arr[i] # get the current cell ECT
    ect_rot1[i,:,:] = np.roll(curr_cell, 8, axis=0) # rotate original ECT and store
    ect_rot2[i,:,:] = np.roll(ect_rot1[i,:,:], 8, axis=0) # rotate from 1st rotation and store
    ect_rot3[i,:,:] = np.roll(ect_rot2[i,:,:], 8, axis=0) # rotate from 2nd rotation and store

ect_rot = np.stack([ect_arr,ect_rot1,ect_rot2,ect_rot3], axis=0) # stack the rotated ect arrays together

print(np.shape(ect_rot)) # print the shape of the rotated ECT array before reshaping

# reshape the ect_rot array so that the samples*rotations is one axis
ect_rot = np.reshape(ect_rot, (np.shape(ect_rot)[0]*np.shape(ect_rot)[1],
                               np.shape(ect_rot)[2],np.shape(ect_rot)[3]))

# flatten the cell ECT data to a 2D array by sample
flat_rot_ect = np.reshape(ect_rot,
                             (np.shape(ect_rot)[0],
                              np.shape(ect_rot)[1]*np.shape(ect_rot)[2])
                            )

# check the shape of the final ect_rot array
np.shape(flat_rot_ect)


Check that the rotation is working by selecting random ECTs to look at

In [ ]:
import random

rands = []
for i in range(4):
    rands.append(random.randint(0,len(ect_arr)-1))

plt.figure(figsize=(4,len(rands)))
counter = 1

for i in range(len(rands)):
    ect0 = ect_arr[rands[i]]
    ect1 = ect_rot1[rands[i]]
    ect2 = ect_rot2[rands[i]]
    ect3 = ect_rot3[rands[i]]
    plt.subplot(len(rands),4,counter)
    plt.imshow(ect0.T)
    plt.axis("off")
    counter+=1
    plt.subplot(len(rands),4,counter)
    plt.imshow(ect1.T)
    plt.axis("off")
    counter+=1
    plt.subplot(len(rands),4,counter)
    plt.imshow(ect2.T)
    plt.axis("off")
    counter+=1
    plt.subplot(len(rands),4,counter)
    plt.imshow(ect3.T)
    plt.axis("off")
    counter+=1

plt.tight_layout()


Create a new dataframe with a `rotation` column with the rotation identity of the corresponding ECT

In [ ]:
# create copy dataframes
rot0_df = pavement_cell_df.copy()
rot1_df = pavement_cell_df.copy()
rot2_df = pavement_cell_df.copy()
rot3_df = pavement_cell_df.copy()

# populate the rotation column
rot0_df["rotation"] = "rotate0"
rot1_df["rotation"] = "rotate1"
rot2_df["rotation"] = "rotate2"
rot3_df["rotation"] = "rotate3"

# concatenate the dataframes together
rotate_df = pd.concat([rot0_df, rot1_df, rot2_df, rot3_df], ignore_index=True)

# check the final dataframe
rotate_df.tail()

In [ ]:
# create the PHATE embedding and transform
phate_operator = phate.PHATE(n_components=3)
phate_rotation = phate_operator.fit_transform(flat_rot_ect)

Add the PHATE axes to the dataframe for plotting

In [ ]:
rotate_df["ROTphate1"] = phate_rotation[:,0]
rotate_df["ROTphate2"] = phate_rotation[:,1]
rotate_df["ROTphate3"] = phate_rotation[:,2]

Head the dataframe so we know what we are working with

In [ ]:
rotate_df.head()

Create a 3D animation by rotation

In [ ]:
sample_labels = rotate_df["rotation"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_rotation, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", filename="rotation_rotation.gif")

Create a 3D animation by width-to-length ratio

In [ ]:
sample_labels = rotate_df["wl_ratio"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_rotation, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", cmap="inferno",filename="rotation_aspect_ratio.gif")

Create a 3D animation by solidity

In [ ]:
sample_labels = rotate_df["solidity8"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_rotation, c=sample_labels, figsize=(8,6), ticks=True, label_prefix="PHATE", cmap="inferno",filename="rotation_solidity.gif")
